<a href="https://colab.research.google.com/github/mmorari-cmyk/CURSO_SE.ALES_Y_SISTEMAS/blob/main/Taller2/desboard1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#instalación de librerías
!pip install streamlit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 61.7 MB/s eta 0:00:00


In [3]:
!mkdir pages

In [21]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile
!pip install pydub
!pip install --upgrade youtube_dl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2025.10.22-py3-none-any.whl size=3025478 sha256=1ed981a4cbe0663cf5e87375be3a1025c898513dee849a10aa6add3666ca10e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-mphfmt02/wheels/b6/70/13/8d2d11b326f983030b72df6408392d5c1b3bc27a9db8b7c5b0
Successfully built yt-dlp
  Attempting uninstall: yt-dlp
    Found existing installation: yt-dlp 2022.4.8
    Uninstalling yt-dlp-2022.4.8:
      Successfully uninstalled yt-dlp-2022.4.8


In [31]:
#Guarda el contenido de esta celda como un archivo 'Modulacion.py'
%%writefile modulacion_AM.py
#La librería principal para crear la interfaz web.

import streamlit as st
#Para manejar los datos de audio como arreglos numéricos.
import numpy as np

# Para generar gráficos.
import matplotlib.pyplot as plt

#Librerías especializadas para leer y escribir archivos de audio, especialmente en formato .wav.
import soundfile as sf

# Para interactuar con el sistema operativo.
import os

 # Librería para descargas
import youtube_dl

from scipy.signal import butter, filtfilt

from scipy.io import wavfile     # Otra librería para manejar archivos .wav.
import subprocess      # Permite ejecutar comandos de terminal desde Python.
from pyngrok import ngrok     # Para exponer la app local a internet

# FUNCIONES

# Descarga el audio de un link usando la herramienta externa 'yt-dlp'.

def download_audio(link, output_name="audio.mp3"):
            command = [
                "yt-dlp",
                "--extract-audio",    # Extrae solo el audio.
                "--audio-format", "mp3",     # Lo convierte a formato mp3.
                "-o", output_name,
                link
            ]
            subprocess.run(command, check=True)


# Convierte el archivo .mp3 a .wav usando la herramienta externa 'ffmpeg'.
def convert_mp3_to_wav(input_file="audio.mp3", output_file="output.wav"):
    command = [
        "ffmpeg",
        "-y",
        "-i", input_file,
        output_file
    ]
    subprocess.run(command, check=True)

#Función alternativa
#def download_ytvid_as_mp3(video_url, name):
    #video_info = youtube_dl.YoutubeDL().extract_info(url=video_url, download=False)
    #filename = f"{name}.mp3"
    #options = {
     #   'format': 'bestaudio/best',
      #  'keepvideo': False,
       # 'outtmpl': filename,
    #}

    #ydl = youtube_dl.YoutubeDL(options)
    #ydl.download([video_info['webpage_url']])

    #return filename


#titulo y texto simple
st.sidebar.title("presentado por:")
st.sidebar.write("Marlyn Nathalia Mora Riascos")

# Frecuencia portadora y mensaje
Fc = 100
Fm = 10
Am = 1

ti = 0
tf = 5 / Fm
Fs = 5 * Fc

t = np.arange(ti, tf, 1 / Fs)
m = Am * np.cos(2 * np.pi * Fm * t)  # Mensaje

Im = 1
Ac = max(abs(m)) / Im  # Amplitud de la portadora
c = Ac * np.cos(2 * np.pi * Fc * t)  # Portadora

# Señal modulada en AM
y = (1 + m / Ac) * c

# Transformadas de Fourier
Xfc = np.fft.fft(c)
Xfm = np.fft.fft(m)
Xfy = np.fft.fft(y)
vfre = np.fft.fftfreq(len(c), 1 / Fs)

# Gráficas en Streamlit
st.title("Modulación AM")
st.markdown("""
La modulación es una técnica esencial en las telecomunicaciones que permite transmitir
        información de manera eficiente a largas distancias. A continuación, se desglosa
        este proceso paso a paso.
        """
    )

st.markdown("---")

st.subheader("1. ¿Por qué es necesaria la modulación?")

st.markdown(
        """
        La información que deseamos enviar (voz, música, datos) se representa con **señales mensaje**,
        que son de baja frecuencia. Estas señales no son aptas para ser transmitidas
        directamente por el aire por dos razones:

        *   **Propagación ineficiente:** Requieren antenas gigantescas y no pueden viajar largas distancias.
        *   **Interferencia:** Múltiples señales se mezclarían, haciendo imposible sintonizar una en específico.

        La solución es "montar" nuestra señal de baja frecuencia en una onda de **alta frecuencia**
        que sí puede viajar eficientemente.
        """
    )

st.markdown("---")

st.subheader("2. Los Componentes Clave")

st.markdown("Para modular, necesitamos dos señales principales:")

st.markdown("- **Señal Mensaje (o Moduladora), $m(t)$:** Es la señal de baja frecuencia que contiene la información (ej. una onda de audio).")
st.latex(r"m(t) = A_m \cos(2\pi F_m t)")

st.markdown("- **Señal Portadora, $c(t)$:** Es una onda de alta frecuencia, constante y sin información, que servirá como \"vehículo\".")
st.latex(r"c(t) = A_c \cos(2\pi F_c t)")

st.markdown("Donde la frecuencia de la portadora es mucho mayor que la del mensaje ($F_c \gg F_m$).")

st.markdown("---")

st.subheader("3. El Mecanismo de la Modulación AM")

st.markdown(
        """
        El proceso consiste en modificar una característica de la señal portadora usando la señal mensaje.
        En el caso específico de la **AM**, la **amplitud** de la portadora ($A_c$) se hace variar
        al ritmo que marca la señal mensaje ($m(t)$).

        El resultado es una nueva señal de alta frecuencia cuya **"envolvente"** (la silueta que
        dibujan sus picos) tiene la misma forma que la señal mensaje original.
        """
    )

st.markdown("---")

st.subheader("4. La Ecuación Matemática de la Modulación AM")

st.markdown("La señal modulada final, $y(t)$, se calcula combinando la portadora y el mensaje de la siguiente forma:")
st.latex(r"y(t) = \left(1 + \frac{m(t)}{A_c}\right) c(t)")

st.markdown("Si sustituimos la fórmula de la portadora $c(t)$, obtenemos la expresión más común:")
st.latex(r"y(t) = A_c \left(1 + \frac{m(t)}{A_c}\right) \cos(2\pi F_c t)")

st.markdown("---")

st.subheader("5. Calibración: El Índice de Modulación ($I_m$)")

st.markdown(
        """
        Para controlar la calidad de la modulación y evitar distorsiones, se utiliza el
        **índice de modulación**. Este parámetro compara qué tan grande es la amplitud del
        mensaje respecto a la amplitud de la portadora.
        """
    )
st.latex(r"I_m = \frac{\text{Peak}\{|m(t)|\}}{A_c}")

st.markdown(
        """
        *   Si **$I_m < 1$ (Submodulación):** La modulación es correcta.
        *   Si **$I_m = 1$ (Modulación al 100%):** Es el punto ideal para máxima eficiencia sin distorsión.
        *   Si **$I_m > 1$ (Sobremodulación):** Ocurre distorsión y la información se corrompe.
        """
    )

st.markdown("---")

st.subheader("6. El Paso Final: La Demodulación")
st.markdown(
        """
        Una vez que la señal modulada $y(t)$ llega a su destino (por ejemplo, una radio),
        el receptor realiza el proceso inverso, llamado **demodulación**. Este proceso consiste
        en extraer la envolvente de la señal recibida para recuperar la señal mensaje original."""
        )

st.subheader("Señales en el tiempo")

# Gráfica de la portadora
st.markdown("""
Gráfica portadora y mensaje
""")
plt.figure()
plt.plot(t, c, label='Portadora')
plt.plot(t, m, label='Mensaje')
plt.xlabel('$t[s]$')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt.gcf())
# Señal modulada
st.subheader("""
Gráfica de la señal modulada
""")
st.markdown("""Se modula la señal a partir de la señal mensaje y señal portadora
""")
plt.figure()
plt.plot(t, y, label='Señal Modulada')
plt.xlabel('$t[s]$')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt.gcf())
st.subheader("A partir de la fft se calcula el espectro de la señal portadora, mensaje y modulada")
st.markdown("""Espectro de portadora""")

# Espectro de la portadora
plt.figure()
plt.plot(vfre, abs(Xfc), label='Portadora')
plt.xlim([0, 2 * Fc])
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())
st.markdown("""Espectro del mensaje""")
# Espectro del mensaje
plt.figure()
plt.plot(vfre, abs(Xfm), label='Mensaje')
plt.xlim([0, 2 * Fc])
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())
st.markdown("""Espectro de señal modulada""")
# Espectro de la señal modulada
plt.figure()
plt.plot(vfre, abs(Xfy), label='Modulado')
plt.xlim([0, 2 * Fc])
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())


st.header("Modulación de Audio desde YouTube")

# Entrada de link de YouTube
link = st.text_input("Ingrese el link del video de YouTube")

# --- Estado para detectar cambio de link y limpiar ---
if 'last_link' not in st.session_state:
    st.session_state.last_link = None
if 'prev_files' not in st.session_state:
    st.session_state.prev_files = []

def _cleanup_old_files():
    import os
    for f in st.session_state.prev_files:
        try:
            if os.path.exists(f):
                os.remove(f)
        except:
            pass
    st.session_state.prev_files = []

if link:
    import hashlib, time, os
    # si cambia el link: limpiar cache y archivos previos
    if link != st.session_state.last_link:
        _cleanup_old_files()
        # por si quedó algo cacheado en otras partes
        try:
            st.cache_data.clear()
        except:
            pass
        st.session_state.last_link = link

    # nombres únicos por link (evita cache del navegador)
    uid = hashlib.md5((link + str(time.time())).encode()).hexdigest()[:8]
    mp3_name = f"audio_{uid}.mp3"
    wav_name = f"output_{uid}.wav"

    with st.spinner("Descargando audio..."):
        download_audio(link, output_name=mp3_name)
        convert_mp3_to_wav(mp3_name, wav_name)

    # registrar para limpieza después
    st.session_state.prev_files.extend([mp3_name, wav_name])

    # Cargar el audio
    audio_file = wav_name
    x, fs = sf.read(audio_file)
    fs = int(fs)  # Asegura entero para el reproductor

    # Selección de fragmento de audio
    st.subheader("Selección de Fragmento de Audio")
    x_use = x[:, 0] if (x.ndim == 2 and x.shape[1] > 0) else x
    duration = len(x_use) / fs
    window = 5

    if duration < window:
        st.warning("El audio dura menos de 5 s; se utilizará todo el audio disponible.")
        start_time = 0
        end_time = duration
    else:
        max_start = max(0, int(duration) - window)
        start_time = st.slider("Tiempo de inicio (segundos)", 0, max_start, 0)
        end_time = start_time + window

    i0 = int(start_time * fs)
    i1 = int(min(end_time * fs, len(x_use)))
    xs = x_use[i0:i1]
    if xs.size == 0:
        xs = np.zeros(1, dtype=np.float32)

    tt = np.arange(0, xs.size) / fs

    # Gráfica del fragmento de audio
    plt.figure()
    plt.plot(tt, xs, label="Canal izquierdo")
    plt.xlabel('$t[s]$')
    plt.ylabel('$x(t)$')
    plt.legend()
    st.pyplot(plt.gcf())

    st.markdown("""
    Si el fragmento de la canción es la señal de mensaje y la señal portadora es:
    """)
    st.latex("c(t)=A_c\\cos(2\\pi F_c t)")
    st.latex(r"F_c=1000\\,\\text{Hz}")

    # Definir índice de modulación
    Im = st.slider("Seleccione el índice de modulación", min_value=0.1, max_value=1.0, value=0.8)

    # Frecuencia portadora (fijada en 1000 Hz)
    Fc = 1000

    # Portadora y modulada del FRAGMENTO
    t = np.arange(0, xs.size) / fs
    peak = float(np.max(np.abs(xs))) if xs.size else 0.0
    Ac = (peak / Im) if peak > 0 else 1.0
    c = Ac * np.sin(2 * np.pi * Fc * t)
    y = (1 + xs / Ac) * c

# Espectro de Frecuencia
Xfc = np.fft.rfft(c)
Xfm = np.fft.rfft(xs)
Xfy = np.fft.rfft(y)
vfre = np.fft.rfftfreq(len(c), 1 / fs)

plt.figure()
plt.plot(t, c, label='portadora')
plt.plot(t, xs, label='mensaje')
plt.xlabel('$t[s]$')
plt.legend()
st.pyplot(plt.gcf())

st.subheader("Señal modulada en AM")
plt.figure()
plt.plot(t, y)
plt.xlabel('$t[s]$')
plt.ylabel('$y(t)$')
st.pyplot(plt.gcf())

st.subheader("Espectro de Frecuencia del Audio Modulado")
st.markdown("""- A continuación, se presenta el espectro de la señal mensaje (canción), portadora y modulada.""")

# Espectro de la portadora
plt.figure()
plt.plot(vfre, np.abs(Xfc), label='Portadora')
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
st.pyplot(plt.gcf())

# Subplots: Modulado y Mensaje
plt.figure(figsize=(8, 6))
plt.subplot(2, 1, 1)
plt.plot(vfre, np.abs(Xfy), label='Modulado', color="r")
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
plt.ylim([np.min(np.abs(Xfy)), 1.1*np.max(np.abs(Xfy))])

plt.subplot(2, 1, 2)
plt.plot(vfre, np.abs(Xfm), label='Mensaje')
plt.xlabel('$f [Hz]$')
plt.ylabel('$|X(f)|$')
plt.legend()
plt.ylim([np.min(np.abs(Xfm)), 1.1*np.max(np.abs(Xfm))])
st.pyplot(plt.gcf())
plt.close()


# Demodulación (recuperar el mensaje)
st.subheader("Demodulación AM (recuperación del mensaje)")

# 1) Mezcla coherente con la misma portadora
y_demod = y * np.cos(2 * np.pi * Fc * t)

# 2) Filtro pasa-bajas para extraer el mensaje (envolvente)
fcorte = Fc / 2.0                       # corte seguro << Fc
Wn = fcorte / (fs / 2.0)                # normalizado a Nyquist
Wn = max(1e-4, min(Wn, 0.99))           # asegurar 0 < Wn < 1

b, a = butter(5, Wn, btype="low")
m_rec = filtfilt(b, a, y_demod)

# 3) Graficar mensaje recuperado
plt.figure()
plt.plot(t, m_rec, label="m̂(t) (recuperado)")
plt.xlabel("$t$ [s]")
plt.ylabel("Amplitud")
plt.title("Mensaje recuperado (demodulación AM)")
plt.legend()
st.pyplot(plt.gcf())

# 4) Breve explicación
st.markdown(r"""
** en AM, la envolvente de la portadora replica la forma del mensaje.
Al multiplicar $y(t)$ por $\cos(2\pi f_c t)$ trasladamos el mensaje a torno de 0 Hz
y luego un **pasa-bajas** elimina las componentes cercanas a $f_c$.
En frecuencia aparecen la portadora en $f_c$ y bandas laterales en $f_c\pm f_m$.
Con audio real (contenido espectral amplio), las bandas laterales son anchas y contienen la información musical.
""")

#para reproducir arrays como audio
def sane(x):
    x = np.asarray(x)
    if x.ndim == 2:
        x = x[:, 0]
    x = np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0).astype("float32")
    m = np.max(np.abs(x)) if x.size else 0.0
    if m > 1.0:
        x = x / m
    return x

# Reproducir el audio (NO usar start_time ni format cuando pasas arrays)
st.header("Reproducción de Audio")
st.subheader("Audio de la portadora")
st.audio(sane(c), sample_rate=fs)

st.subheader("Audio Original")
st.audio(sane(xs), sample_rate=fs)

st.subheader("Audio Modulado")
st.audio(sane(y), sample_rate=fs)


Overwriting modulacion_AM.py


In [32]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run modulacion_AM.py &>/content/logs.txt & #Cambiar  ...py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-11-09 05:45:25--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-11-09 05:45:25--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-09T06%3A22%3A12Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-09